In [1]:
import numpy as np
from pyscf import gto, scf, df, ao2mo

In [71]:
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='STO-3G', charge = 1)
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='pc-3', charge = 1)
mol = gto.M(atom='Be 0 0 0', basis='6-31G', charge = 0)
#mol = gto.M(atom='H 0 0 0; H 0 0 0.740848149', basis='STO-3G', charge = 0)
#mol = gto.M(atom="O 0.00000 0.00000 0.11779; H 0.00000 0.75545 -0.47116; H 0.00000 -0.75545 -0.47116", basis = "sto-3g")
#mol = gto.M(atom="H 0.8668114 0.6014355  -0.000000000000; O 0.00000 -0.075791807925  0.00000; H -0.8668114 0.6014355  -0.000000000000", basis = "sto-3g")

In [72]:
#calculate nuclear repulsion energy
tot_nuc_energy=0
for i in range(0,len(mol.atom_coords())-1):
    for j in range(0,len(mol.atom_coords())):
        if j > i:
            print("i",i,"-- j",j)
            x=mol.atom_coord(i)-mol.atom_coord(j)
           # print("x",x)
            norm=np.linalg.norm(x)
           # print("norm",norm)
            nuc_energy=mol.atom_charge(i)*mol.atom_charge(j)/norm
            print("nuc_energy",nuc_energy)     
            tot_nuc_energy+=nuc_energy
print("tot_nuc_energy",tot_nuc_energy)

tot_nuc_energy 0


In [73]:
#get integrals and calculate core Hamiltonian

overlap_integrals = mol.intor('int1e_ovlp_sph')
kinetic_energy_integrals = mol.intor('int1e_kin_sph')
nuclear_attraction_integrals = mol.intor('int1e_nuc_sph')
two_electron_integrals = mol.intor('int2e_sph')
H_core = kinetic_energy_integrals + nuclear_attraction_integrals


In [74]:
#diagonalize overlap matrix
overlap_diag_procedure = np.linalg.eigh(overlap_integrals)
#get U and S^-0.5
U = overlap_diag_procedure[1]
s_inverse_sqrt = np.diag(1/overlap_diag_procedure[0])**0.5
#get transformation matrix X
X = np.matmul(U,np.matmul(s_inverse_sqrt,np.matrix.transpose(U)))
#set guess for P matrix, default core hamiltonian
P = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

#start iterations - everything in Mulliken ordering!!!!
counter = 0
maxiter = 100
energy_difference=10000
energy_last_iter=10000
energy=0

for counter in range(0,maxiter):
    G = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    G[i,j]+=P[k,l]*(two_electron_integrals[i,j,k,l]-0.5*two_electron_integrals[i,l,k,j])

    F = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    F = H_core + G
    
    if counter >= 1:
        energy = 0
        for i in range(0,overlap_integrals.shape[0]):
            for j in range(0,overlap_integrals.shape[0]):
                energy += 0.5*(P[i,j]*(H_core[i,j]+F[i,j]))
        energy_difference = abs(energy-energy_last_iter)
        print('iteration {:<9}'.format(counter),"energy = {:<25}".format(energy), "difference = {:<23}".format(energy_difference))
        energy_last_iter = energy
    if energy_difference < 10**-12:
        print("energy converged")
        SCF_final_energy = energy + tot_nuc_energy
        print("final SCF energy including nuclear repulsion:",SCF_final_energy)
        break
    counter+=1    

    F_prime = np.matmul(np.matrix.transpose(X),np.matmul(F,X))
    F_diag_procedure = np.linalg.eigh(F_prime)
    
    C = np.matmul(X,F_diag_procedure[1])
    
    P_new = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    
    for i in range(0,overlap_integrals.shape[0]):
        for j in range(0,overlap_integrals.shape[1]):    
            for a in range(0,mol.nelectron//2):
                P_new[i,j]+=2*C[i,a]*C[j,a]

    P = P_new


iteration 1         energy = -14.354283984261523       difference = 10014.354283984261     
iteration 2         energy = -14.553802777874642       difference = 0.19951879361311953    
iteration 3         energy = -14.566109735589508       difference = 0.012306957714866229   
iteration 4         energy = -14.56673215621623        difference = 0.0006224206267209098  
iteration 5         energy = -14.56676249189739        difference = 3.033568115995422e-05  
iteration 6         energy = -14.56676395907595        difference = 1.4671785599773557e-06 
iteration 7         energy = -14.56676402991702        difference = 7.084106989907468e-08  
iteration 8         energy = -14.566764033336243       difference = 3.419224015033251e-09  
iteration 9         energy = -14.566764033501256       difference = 1.6501289223924687e-10 
iteration 10        energy = -14.56676403350922        difference = 7.963407711031323e-12  
iteration 11        energy = -14.5667640335096         difference = 3.8014036363

In [75]:
#transform to MOs

C_ao_mo = np.zeros((F_diag_procedure[1].shape[0],F_diag_procedure[1].shape[1]))
C_ao_mo[:,:]=C[:,:]

integrals_ijks = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_ijrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_iqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_pqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))

for s in range(0,integrals_ijks.shape[0]):
    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    integrals_ijks[i,j,k,s]+=C_ao_mo[l,s]*two_electron_integrals[i,j,k,l]
    
for s in range(0,integrals_ijrs.shape[0]):
    for r in range(0,integrals_ijrs.shape[1]):
        for i in range(0,two_electron_integrals.shape[0]):
            for j in range(0,two_electron_integrals.shape[1]):
                for k in range(0,two_electron_integrals.shape[2]):
                    integrals_ijrs[i,j,r,s]+=C_ao_mo[k,r]*integrals_ijks[i,j,k,s]

for s in range(0,integrals_iqrs.shape[0]):
    for r in range(0,integrals_iqrs.shape[1]):
        for q in range(0,integrals_iqrs.shape[2]):
            for i in range(0,two_electron_integrals.shape[0]):
                for j in range(0,two_electron_integrals.shape[1]):
                    integrals_iqrs[i,q,r,s]+=C_ao_mo[j,q]*integrals_ijrs[i,j,r,s]

for s in range(0,integrals_pqrs.shape[0]):
    for r in range(0,integrals_pqrs.shape[1]):
        for q in range(0,integrals_pqrs.shape[2]):
            for p in range(0,integrals_pqrs.shape[3]):
                for i in range(0,two_electron_integrals.shape[0]):
                    integrals_pqrs[p,q,r,s]+=C_ao_mo[i,p]*integrals_iqrs[i,q,r,s]
                    

#transform to MO-spin integrals
#DIRAC-Notation!!!!

mo_integrals = integrals_pqrs
mo_integrals_spin = np.zeros((len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals)*2))
for p in range(1,len(mo_integrals_spin)+1):
    for r in range(1,len(mo_integrals_spin)+1):
        if (p%2==r%2):
            #print("pr:", p,r)
            for q in range(1,len(mo_integrals_spin)+1):
                for s in range(1,len(mo_integrals_spin)+1):
                    if (q%2==s%2):
                        #print("qs:", q,s)
                        mo_integrals_spin[p-1,q-1,r-1,s-1]=mo_integrals[(p+1)//2-1,(r+1)//2-1,(q+1)//2-1,(s+1)//2-1]


In [76]:
#get A and B matrizes

eigenenergies=np.zeros((len(F_diag_procedure[0])*2))
for i in range(0,len(eigenenergies)):
    eigenenergies[i]+= np.dot(C.T,np.dot(F,C))[i//2][i//2]

A = np.zeros((mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron),mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron)))
B = np.zeros((mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron),mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron)))

arindex = -1
for a in range(0,mol.nelectron):
    for r in range(mol.nelectron,overlap_integrals.shape[0]*2):
        arindex += 1
        bsindex = -1
        for b in range(0,mol.nelectron):
            for s in range(mol.nelectron,overlap_integrals.shape[0]*2):
                bsindex += 1
                B[arindex,bsindex]+=mo_integrals_spin[r,s,a,b]-mo_integrals_spin[r,s,b,a]
                A[arindex,bsindex]+=mo_integrals_spin[r,b,a,s]-mo_integrals_spin[r,b,s,a]
                if a==b and r==s:
                    A[arindex,bsindex] += eigenenergies[r] - eigenenergies[a]
                    
mat = np.matmul((A+B),(A-B)) 
Exc = np.real(np.sort(np.sqrt(np.linalg.eigvals(mat))))
#print('hartree {:<20}'.format(""),"eV")
#print("-------------------------------------------------------------")
#for i in range(0,len(Exc)): 
#    print('{:<28}'.format(Exc[i]),Exc[i]*27.2116)
            
Exc_cut = unique = np.unique(Exc.round(decimals=8))
Exc_eV = Exc_cut*27.2116
print('hartree {:<21}'.format(""),"eV")
print("-------------------------------------------------------------")
for i in range(0,len(Exc_cut)): 
    print('{:<28}'.format(f'{Exc_cut[i]:11.8f}'),f'{Exc_eV[i].round(decimals=8):12.8f}')

hartree                       eV
-------------------------------------------------------------
 0.00000000                    0.00000000
 0.18956763                    5.15843852
 0.43266264                   11.77344269
 0.43310437                   11.78546287
 0.47989224                   13.05863568
 0.52851627                   14.38177333
 4.30472019                  117.13832392
 4.34055924                  118.11356182
 4.70613586                  128.06148657
 4.72101204                  128.46629123
 4.73163418                  128.75533665
 4.76236572                  129.59159103
